In [98]:
import json
import numpy as np
import math
import os
import copy
from builelib.make_inputdata import make_jsondata_from_Ver2_sheet
# import matplotlib.pyplot as plt

import sys
import builelib.commons as bc
import builelib.climate as climate
import builelib.shading as shading
sys.path.append(os.getcwd())
# データベースファイルの保存場所
database_directory = os.getcwd() + "/database/"
# 気象データファイルの保存場所
climatedata_directory = os.getcwd() + "/climatedata/"


In [99]:
inputfile_name = "../sample/WEBPRO_inputSheet_sample.xlsm"  # デフォルトのファイル名
inputdata, validation = make_jsondata_from_Ver2_sheet(inputfile_name)

In [100]:
def count_Matrix(x, mxL):
    """
    負荷率 X がマトリックス mxL の何番目（ix）のセルに入るかをカウント
    """

    # 初期値
    ix = 0

    # C#の処理に合わせる（代表負荷率にする）
    # 負荷率1.00の場合は x=1.05となるため過負荷判定
    x = math.floor(x*10)/10+0.05

    # 該当するマトリックスを探査
    while x > mxL[ix]:
        ix += 1

        if ix == len(mxL)-1:
            break

    return ix+1


def air_enthalpy(Tdb, X):
    """
    空気のエンタルピーを算出する関数
    (WEBPROに合わせる)
    """

    Ca = 1.006  # 乾き空気の定圧比熱 [kJ/kg･K]
    Cw = 1.805  # 水蒸気の定圧比熱 [kJ/kg･K]
    Lw = 2502   # 水の蒸発潜熱 [kJ/kg]

    if len(Tdb) != len(X):
        raise Exception('温度と湿度のリストの長さが異なります。')
    else:

        H = np.zeros(len(Tdb))
        for i in range(0, len(Tdb)):
            H[i] = (Ca*Tdb[i] + (Cw*Tdb[i]+Lw)*X[i])

    return H

In [101]:

inputdata["PUMP"] = {}
inputdata["REF"] = {}

# 計算結果を格納する変数
resultJson = {

    "設計一次エネルギー消費量[MJ/年]": 0,    # 空調設備の設計一次エネルギー消費量 [MJ/年]
    "基準一次エネルギー消費量[MJ/年]": 0,    # 空調設備の基準一次エネルギー消費量 [MJ/年]
    "設計一次エネルギー消費量[GJ/年]": 0,    # 空調設備の設計一次エネルギー消費量 [GJ/年]
    "基準一次エネルギー消費量[GJ/年]": 0,    # 空調設備の基準一次エネルギー消費量 [GJ/年]
    "設計一次エネルギー消費量[MJ/m2年]": 0,  # 空調設備の設計一次エネルギー消費量 [MJ/年]
    "基準一次エネルギー消費量[MJ/m2年]": 0,  # 空調設備の基準一次エネルギー消費量 [MJ/年]
    "計算対象面積": 0,
    "BEI/AC": 0,
    "Qroom": {
    },
    "AHU":{
    },
    "PUMP":{
    },
    "REF":{
    },
    "年間エネルギー消費量":{
        "空調機群ファン[MWh]": 0,     # 空調機群の一次エネルギー消費量 [MWh]
        "空調機群ファン[GJ]": 0,     # 空調機群の一次エネルギー消費量 [GJ]
        "空調機群全熱交換器[MWh]": 0,  # 全熱交換器の一次エネルギー消費量 [MWh]
        "空調機群全熱交換器[GJ]": 0,  # 全熱交換器の一次エネルギー消費量 [GJ]
        "二次ポンプ群[MWh]": 0,    # 二次ポンプ群の一次エネルギー消費量 [MWh]
        "二次ポンプ群[GJ]": 0,    # 二次ポンプ群の一次エネルギー消費量 [GJ]
        "熱源群熱源主機[MJ]": 0, # 熱源群主機の一次エネルギー消費量 [MJ]
        "熱源群熱源主機[GJ]": 0, # 熱源群主機の一次エネルギー消費量 [GJ]
        "熱源群熱源補機[MWh]": 0,   # 熱源群補機の一次エネルギー消費量 [MWh]
        "熱源群熱源補機[GJ]": 0,     # 熱源群補機の一次エネルギー消費量 [GJ]
        "熱源群一次ポンプ[MWh]": 0,   # 熱源群一次ポンプの一次エネルギー消費量 [MWh]
        "熱源群一次ポンプ[GJ]": 0,    # 熱源群一次ポンプの一次エネルギー消費量 [GJ]
        "熱源群冷却塔ファン[MWh]": 0,  # 熱源群冷却塔ファンの一次エネルギー消費量 [MWh]
        "熱源群冷却塔ファン[GJ]": 0,   # 熱源群冷却塔ファンの一次エネルギー消費量 [GJ]
        "熱源群冷却水ポンプ[MWh]": 0,  # 熱源群冷却水ポンプの一次エネルギー消費量 [MWh]
        "熱源群冷却水ポンプ[GJ]": 0,   # 熱源群冷却水ポンプの一次エネルギー消費量 [GJ]
    },
    "日別エネルギー消費量":{
        "E_fan_MWh_day": np.zeros(365),
        "E_pump_MWh_day": np.zeros(365),
        "E_ref_main_MWh_day": np.zeros(365),
        "E_ref_sub_MWh_day": np.zeros(365),
    },
    "Matrix":{
    },
    "for_CGS":{
    }
}

##----------------------------------------------------------------------------------
## 定数の設定
##----------------------------------------------------------------------------------
k_heatup = 0.84       # ファン・ポンプの発熱比率
Cw = 4.186            # 水の比熱 [kJ/kg・K]
divL = 11             # 負荷帯マトリックス分割数 （10区分＋過負荷1区分）
divT =  6             # 外気温度帯マトリックス分割数


##----------------------------------------------------------------------------------
## データベースファイルの読み込み
##----------------------------------------------------------------------------------

# 流量制御
with open(database_directory + 'FLOWCONTROL.json', 'r', encoding='utf-8') as f:
    FLOWCONTROL = json.load(f)

# 熱源機器特性
with open(database_directory + "HeatSourcePerformance.json", 'r', encoding='utf-8') as f:
    HeatSourcePerformance = json.load(f)

##----------------------------------------------------------------------------------
## 任意評定 （SP-1: 流量制御)
##----------------------------------------------------------------------------------

# 任意評定用の入力があれば追加
if "SpecialInputData" in inputdata:
    if "flow_control" in inputdata["SpecialInputData"]:
        FLOWCONTROL.update(inputdata["SpecialInputData"]["flow_control"])

##----------------------------------------------------------------------------------
## 任意評定 （SP-2：　熱源機器特性)
##----------------------------------------------------------------------------------

# 任意評定用の入力があれば追加
if "SpecialInputData" in inputdata:
    if "heatsource_performance" in inputdata["SpecialInputData"]:
        HeatSourcePerformance.update(inputdata["SpecialInputData"]["heatsource_performance"])


##----------------------------------------------------------------------------------
## マトリックスの設定
##----------------------------------------------------------------------------------

# 地域別データの読み込み
with open(database_directory + 'AREA.json', 'r', encoding='utf-8') as f:
    Area = json.load(f)

# 負荷率帯マトリックス mxL = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.2])
mxL = np.arange(1/(divL-1), 1.01, 1/(divL-1))
mxL = np.append(mxL,1.2)

# 負荷率帯マトリックス（平均） aveL = array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.2 ])
aveL = np.zeros(len(mxL))
for iL in range(0,len(mxL)):
    if iL == 0:
        aveL[0] = mxL[0]/2
    elif iL == len(mxL)-1:
        aveL[iL] = 1.2
    else:
        aveL[iL] = mxL[iL-1] + (mxL[iL]-mxL[iL-1])/2

##----------------------------------------------------------------------------------
## 日平均外気温 （解説書 2.7.4.1）
##----------------------------------------------------------------------------------

# 外気温度帯の上限・下限
mxTH_min = Area[inputdata["Building"]["Region"]+"地域"]["暖房時外気温下限"]
mxTH_max = Area[inputdata["Building"]["Region"]+"地域"]["暖房時外気温上限"]
mxTC_min = Area[inputdata["Building"]["Region"]+"地域"]["冷房時外気温下限"]
mxTC_max = Area[inputdata["Building"]["Region"]+"地域"]["冷房時外気温上限"]

delTC = (mxTC_max - mxTC_min)/divT
delTH = (mxTH_max - mxTH_min)/divT

mxTC = np.arange(mxTC_min+delTC, mxTC_max+delTC, delTC)
mxTH = np.arange(mxTH_min+delTH, mxTH_max+delTH, delTH)

ToadbC = mxTC - delTC/2
ToadbH = mxTH - delTH/2

# 保存用
resultJson["Matrix"]["ToadbC"] = ToadbC
resultJson["Matrix"]["ToadbH"] = ToadbH

##----------------------------------------------------------------------------------
## 他人から供給された熱の一次エネルギー換算係数（デフォルト）
##----------------------------------------------------------------------------------

if inputdata["Building"]["Coefficient_DHC"]["Cooling"] == None:
    inputdata["Building"]["Coefficient_DHC"]["Cooling"] = 1.36

if inputdata["Building"]["Coefficient_DHC"]["Heating"] == None:
    inputdata["Building"]["Coefficient_DHC"]["Heating"] = 1.36


##----------------------------------------------------------------------------------
## 気象データ（解説書 2.2.1）
## 任意評定 （SP-5: 気象データ)
##----------------------------------------------------------------------------------

if "climate_data" in inputdata["SpecialInputData"]:  # 任意入力（SP-5）

    # 外気温 [℃]
    ToutALL = np.array(inputdata["SpecialInputData"]["climate_data"]["Tout"])
    # 外気湿度 [kg/kgDA]
    XoutALL = np.array(inputdata["SpecialInputData"]["climate_data"]["Xout"])
    # 法線面直達日射量 [W/m2]
    IodALL  = np.array(inputdata["SpecialInputData"]["climate_data"]["Iod"])
    # 水平面天空日射量 [W/m2]
    IosALL  = np.array(inputdata["SpecialInputData"]["climate_data"]["Ios"])
    # 水平面夜間放射量 [W/m2]
    InnALL  = np.array(inputdata["SpecialInputData"]["climate_data"]["Inn"])

else:

    # 気象データ（HASP形式）読み込み ＜365×24の行列＞
    [ToutALL, XoutALL, IodALL, IosALL, InnALL] = \
        climate.readHaspClimateData( climatedata_directory + "/C1_" + Area[inputdata["Building"]["Region"]+"地域"]["気象データファイル名"] )

# 緯度
phi  = Area[inputdata["Building"]["Region"]+"地域"]["緯度"]
# 経度
longi  = Area[inputdata["Building"]["Region"]+"地域"]["経度"]

##----------------------------------------------------------------------------------
## 冷暖房期間（解説書 2.2.2）
##----------------------------------------------------------------------------------

# 空調運転モード
with open(database_directory + 'ACoperationMode.json', 'r', encoding='utf-8') as f:
    ACoperationMode = json.load(f)

# 各日の冷暖房期間の種類（冷房期、暖房期、中間期）（365×1の行列）
ac_mode = ACoperationMode[ Area[inputdata["Building"]["Region"]+"地域"]["空調運転モードタイプ"] ]


##----------------------------------------------------------------------------------
## 平均外気温（解説書 2.2.3）
##----------------------------------------------------------------------------------

# 日平均外気温[℃]（365×1）
Toa_ave = np.mean(ToutALL,1)
Toa_day = np.mean(ToutALL[:,[6,7,8,9,10,11,12,13,14,15,16,17]],1)
Toa_ngt = np.mean(ToutALL[:,[0,1,2,3,4,5,18,19,20,21,22,23]],1)

# 日平均外気絶対湿度 [kg/kgDA]（365×1）
Xoa_ave = np.mean(XoutALL,1)
Xoa_day = np.mean(XoutALL[:,[6,7,8,9,10,11,12,13,14,15,16,17]],1)
Xoa_ngt = np.mean(XoutALL[:,[0,1,2,3,4,5,18,19,20,21,22,23]],1)


##----------------------------------------------------------------------------------
## 外気エンタルピー（解説書 2.2.4）
##----------------------------------------------------------------------------------

Hoa_ave = air_enthalpy(Toa_ave, Xoa_ave)
Hoa_day = air_enthalpy(Toa_day, Xoa_day)
Hoa_ngt = air_enthalpy(Toa_ngt, Xoa_ngt)


##----------------------------------------------------------------------------------
## 空調室の設定温度、室内エンタルピー（解説書 2.3.1、2.3.2）
##----------------------------------------------------------------------------------

TroomSP = np.zeros(365)    # 室内設定温度
RroomSP = np.zeros(365)    # 室内設定湿度
Hroom   = np.zeros(365)    # 室内設定エンタルピー

for dd in range(0,365):

    if ac_mode[dd] == "冷房":
        TroomSP[dd] = 26
        RroomSP[dd] = 50
        Hroom[dd] = 52.91

    elif ac_mode[dd] == "中間":
        TroomSP[dd] = 24
        RroomSP[dd] = 50
        Hroom[dd] = 47.81

    elif ac_mode[dd] == "暖房":
        TroomSP[dd] = 22
        RroomSP[dd] = 40
        Hroom[dd] = 38.81


##----------------------------------------------------------------------------------
## 任意評定 （SP-6: カレンダーパターン)
##----------------------------------------------------------------------------------
input_calendar = []
if "calender" in inputdata["SpecialInputData"]:
    input_calendar = inputdata["SpecialInputData"]["calender"]


##----------------------------------------------------------------------------------
## 空調機の稼働状態、内部発熱量（解説書 2.3.3、2.3.4）
##----------------------------------------------------------------------------------

roomAreaTotal = 0
roomScheduleRoom   = {}
roomScheduleLight  = {}
roomSchedulePerson = {}
roomScheduleOAapp  = {}
roomDayMode        = {}

# 空調ゾーン毎にループ
for room_zone_name in inputdata["AirConditioningZone"]:

    if room_zone_name in inputdata["Rooms"]:  # ゾーン分けがない場合

        # 建物用途・室用途、ゾーン面積等の取得
        inputdata["AirConditioningZone"][room_zone_name]["buildingType"]  = inputdata["Rooms"][room_zone_name]["buildingType"]
        inputdata["AirConditioningZone"][room_zone_name]["roomType"]      = inputdata["Rooms"][room_zone_name]["roomType"]
        inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]      = inputdata["Rooms"][room_zone_name]["roomArea"]
        inputdata["AirConditioningZone"][room_zone_name]["ceilingHeight"] = inputdata["Rooms"][room_zone_name]["ceilingHeight"]

    else:

        # 各室のゾーンを検索
        for room_name in inputdata["Rooms"]:
            if inputdata["Rooms"][room_name]["zone"] != None:   # ゾーンがあれば
                for zone_name  in inputdata["Rooms"][room_name]["zone"]:   # ゾーン名を検索
                    if room_zone_name == (room_name+"_"+zone_name):

                        inputdata["AirConditioningZone"][room_zone_name]["buildingType"]  = inputdata["Rooms"][room_name]["buildingType"]
                        inputdata["AirConditioningZone"][room_zone_name]["roomType"]      = inputdata["Rooms"][room_name]["roomType"]
                        inputdata["AirConditioningZone"][room_zone_name]["ceilingHeight"] = inputdata["Rooms"][room_name]["ceilingHeight"]
                        inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]      = inputdata["Rooms"][room_name]["zone"][zone_name]["zoneArea"]

                        break

    # 365日×24時間分のスケジュール （365×24の行列を格納した dict型）
    roomScheduleRoom[room_zone_name], roomScheduleLight[room_zone_name], roomSchedulePerson[room_zone_name], roomScheduleOAapp[room_zone_name], roomDayMode[room_zone_name] = \
        bc.get_roomUsageSchedule(inputdata["AirConditioningZone"][room_zone_name]["buildingType"], inputdata["AirConditioningZone"][room_zone_name]["roomType"], input_calendar)


    # 空調対象面積の合計
    roomAreaTotal += inputdata["AirConditioningZone"][room_zone_name]["zoneArea"]


##----------------------------------------------------------------------------------
## 任意評定 （SP-7: 室スケジュール)
##----------------------------------------------------------------------------------

if "room_schedule" in inputdata["SpecialInputData"]:

    # 空調ゾーン毎にループ
    for room_zone_name in inputdata["AirConditioningZone"]:

        # SP-7に入力されていれば
        if room_zone_name in inputdata["SpecialInputData"]["room_schedule"]:

            # 使用時間帯
            roomDayMode[room_zone_name] = inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["roomDayMode"]

            if "室の同時使用率" in inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]:
                roomScheduleRoom_tmp = np.array(inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]["室の同時使用率"]).astype("float")
                roomScheduleRoom_tmp = np.where(roomScheduleRoom_tmp < 1, 0, roomScheduleRoom_tmp)  # 同時使用率は考えない
                roomScheduleRoom[room_zone_name] = roomScheduleRoom_tmp
            if "照明発熱密度比率" in inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]:
                roomScheduleLight[room_zone_name] = np.array(inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]["照明発熱密度比率"])
            if "人体発熱密度比率" in inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]:
                roomSchedulePerson[room_zone_name] = np.array(inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]["人体発熱密度比率"])
            if "機器発熱密度比率" in inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]:
                roomScheduleOAapp[room_zone_name] = np.array(inputdata["SpecialInputData"]["room_schedule"][room_zone_name]["schedule"]["機器発熱密度比率"])


#%%
##----------------------------------------------------------------------------------
## 室負荷計算（解説書 2.4）
##----------------------------------------------------------------------------------

for room_zone_name in inputdata["AirConditioningZone"]:

    resultJson["Qroom"][room_zone_name] = {
        "Qwall_T": np.zeros(365),  # 壁からの温度差による熱取得 [W/m2]
        "Qwall_S": np.zeros(365),  # 壁からの日射による熱取得 [W/m2]
        "Qwall_N": np.zeros(365),  # 壁からの夜間放射による熱取得（マイナス）[W/m2]
        "Qwind_T": np.zeros(365),  # 窓からの温度差による熱取得 [W/m2]
        "Qwind_S": np.zeros(365),  # 窓からの日射による熱取得 [W/m2]
        "Qwind_N": np.zeros(365),  # 窓からの夜間放射による熱取得（マイナス）[W/m2]
        "QroomDc": np.zeros(365),  # 冷房熱取得（日積算）　[MJ/day]
        "QroomDh": np.zeros(365),   # 暖房熱取得（日積算）　[MJ/day]
        "QroomHc": np.zeros((365,24)),  # 冷房熱取得（時刻別）　[MJ/h]
        "QroomHh": np.zeros((365,24))   # 暖房熱取得（時刻別）　[MJ/h]
    }

##----------------------------------------------------------------------------------
## 外皮面への入射日射量（解説書 2.4.1）
##----------------------------------------------------------------------------------

solor_radiation = {
    "直達":{
    },
    "直達_入射角特性込":{
    },
    "天空":{
    },
    "夜間":{
    }
}

# 方位角別の日射量
(solor_radiation["直達"]["南"],  solor_radiation["直達_入射角特性込"]["南"], solor_radiation["天空"]["垂直"], solor_radiation["夜間"]["垂直"])  = \
    climate.solarRadiationByAzimuth(  0, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["南西"], solor_radiation["直達_入射角特性込"]["南西"], _, _) = climate.solarRadiationByAzimuth( 45, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["西"],  solor_radiation["直達_入射角特性込"]["西"], _, _)  = climate.solarRadiationByAzimuth( 90, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北西"], solor_radiation["直達_入射角特性込"]["北西"], _, _) = climate.solarRadiationByAzimuth(135, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北"],  solor_radiation["直達_入射角特性込"]["北"], _, _)  = climate.solarRadiationByAzimuth(180, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["北東"], solor_radiation["直達_入射角特性込"]["北東"], _, _) = climate.solarRadiationByAzimuth(225, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["東"],  solor_radiation["直達_入射角特性込"]["東"], _, _)  = climate.solarRadiationByAzimuth(270, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["南東"], solor_radiation["直達_入射角特性込"]["南東"], _, _) = climate.solarRadiationByAzimuth(315, 90, phi, longi, IodALL, IosALL, InnALL)
(solor_radiation["直達"]["水平"], solor_radiation["直達_入射角特性込"]["水平"], solor_radiation["天空"]["水平"], solor_radiation["夜間"]["水平"])  = \
    climate.solarRadiationByAzimuth(  0,  0, phi, longi, IodALL, IosALL, InnALL)


In [102]:

##----------------------------------------------------------------------------------
## 外壁等の熱貫流率の算出（解説書 附属書A.1）
##----------------------------------------------------------------------------------

### ISSUE : 二つのデータベースにわかれてしまっているので統一する。###

# 標準入力法建材データの読み込み
with open(database_directory + 'HeatThermalConductivity.json', 'r', encoding='utf-8') as f:
    HeatThermalConductivity = json.load(f)

# モデル建物法建材データの読み込み
with open(database_directory + 'HeatThermalConductivity_model.json', 'r', encoding='utf-8') as f:
    HeatThermalConductivity_model = json.load(f)

for wall_name in inputdata["WallConfigure"].keys():
    Rvalue = 0.11 + 0.04

    for layer in enumerate(inputdata["WallConfigure"][wall_name]["layers"]):
        if (layer[1]["materialID"] == "密閉中空層") or (layer[1]["materialID"] == "非密閉中空層"):

            # 空気層の場合
            Rvalue += HeatThermalConductivity[layer[1]["materialID"]]["熱抵抗値"]

        else:
            # 空気層以外の断熱材を指定している場合
            if layer[1]["thickness"] != None:
                material_name = layer[1]["materialID"].replace('\u3000', '')
                Rvalue += (layer[1]["thickness"]/1000) / HeatThermalConductivity[material_name]["熱伝導率"]

    inputdata["WallConfigure"][wall_name]["Uvalue"] = 1/Rvalue
    print(Rvalue)


1.9196590909090911
1.0194726107226106
0.2775394736842105


In [103]:
inputdata["WallConfigure"][wall_name]["inputMethod"]
inputdata["WallConfigure"].keys()

dict_keys(['R1', 'W1', 'FG1'])

In [104]:
# Creating the list of keys from HeatThermalConductivity
material_keys = list(HeatThermalConductivity.keys())

# Extracting a list of "熱伝導率" or "熱伝達率" if present
conductivity_list = [material.get("熱伝導率", None) or material.get("熱伝達率", None) for material in HeatThermalConductivity.values()]

# Creating the binary array
Bi_X = [0] * len(material_keys)

for wall_name in inputdata["WallConfigure"].keys():
    Bi_X = [0] * len(material_keys)
    Rvalue = 0.11 + 0.04
    for layer in inputdata["WallConfigure"][wall_name]["layers"]:
        material_id = layer["materialID"].replace("\u3000", "")
        if material_id in material_keys:
            index = material_keys.index(material_id)
            thickness = layer.get("thickness")
            if thickness is not None:  # Check if thickness is not None
                Bi_X[index] = thickness / 1000
            else:
                Bi_X[index] = 1

    for i in range(len(material_keys)):
        if Bi_X[i] != 0:
            if i == 85:
                Rvalue += 0.15
            elif i == 86:
                Rvalue += 0.09
            else:
                Rvalue += Bi_X[i] / conductivity_list[i]

    inputdata["WallConfigure"][wall_name]["Uvalue"] = 1 / Rvalue


In [105]:
# Creating the list of keys from HeatThermalConductivity
material_keys = list(HeatThermalConductivity.keys())

# Extracting a list of "熱伝導率" or "熱伝達率" if present
conductivity_list = [material.get("熱伝導率", None) or material.get("熱伝達率", None) for material in HeatThermalConductivity.values()]

# Creating a 2D array for Bi_X
Bi_X_dict = {wall_name: np.zeros(len(material_keys)) for wall_name in inputdata["WallConfigure"].keys()}

for wall_name in inputdata["WallConfigure"].keys():
    Rvalue = 0.11 + 0.04
    for layer in inputdata["WallConfigure"][wall_name]["layers"]:
        material_id = layer["materialID"].replace("\u3000", "")
        if material_id in material_keys:
            index = material_keys.index(material_id)
            thickness = layer.get("thickness")
            if thickness is not None:  # Check if thickness is not None
                Bi_X_dict[wall_name][index] = thickness / 1000
            else:
                Bi_X_dict[wall_name][index] = 1

    for i in range(len(material_keys)):
        if Bi_X_dict[wall_name][i] != 0:
            if i == 85:
                Rvalue += 0.15
            elif i == 86:
                Rvalue += 0.09
            else:
                Rvalue += Bi_X_dict[wall_name][i] / conductivity_list[i]

    inputdata["WallConfigure"][wall_name]["Uvalue"] = 1 / Rvalue

# Displaying the Bi_X array for each wall
for wall_name in Bi_X_dict:
    print(f"Bi_X for {wall_name}: {Bi_X_dict[wall_name]}")

# Displaying the U-values for verification
for wall_name in inputdata["WallConfigure"]:
    print(f"{wall_name} Uvalue: {inputdata['WallConfigure'][wall_name]['Uvalue']}")

Bi_X for R1: [0.    0.    0.    0.    0.    0.    0.06  0.    0.    0.    0.    0.
 0.015 0.    0.    0.01  0.    0.    0.    0.    0.    0.    0.    0.012
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.005 0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.05  0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    1.   ]
Bi_X for W1: [0.    0.    0.    0.    0.    0.    0.15  0.    0.    0.    0.    0.
 0.025 0.    0.    0.008 0.    0.    0.    0.    0.01  0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.025 0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.  

In [106]:
import random

material_keys = list(HeatThermalConductivity_model.keys())

# HeatThermalConductivity_model から値を取得してリストに変換
conductivity_list = list(HeatThermalConductivity_model.values())

Bi_X = [0] * len(material_keys)
Rvalue = 0

# 1個以上10個未満のランダムな数の要素を1に設定
num_elements_to_set = random.randint(1, 9)
indices_to_set = random.sample(range(len(material_keys)), num_elements_to_set)

for index in indices_to_set:
    Bi_X[index] = 1

for i in range(len(material_keys)):
    if Bi_X[i] == 1:
        Rvalue += conductivity_list[i]

print(Rvalue)

0.165


In [107]:
import difflib
def find_closest_material(material, dataset_keys):
    closest_match = difflib.get_close_matches(material, dataset_keys, n=1, cutoff=0.1)
    return closest_match[0] if closest_match else None

def map_u_values_to_list(material_keys, u_value_dataset):
    u_value_list = []
    for material in material_keys:
        closest_material = find_closest_material(material, u_value_dataset.keys())
        if closest_material:
            u_value_list.append(u_value_dataset[closest_material])
        else:
            u_value_list.append(None)  # Or some default value or handling
    return u_value_list

u_value_list = map_u_values_to_list(material_keys, HeatThermalConductivity_model)

# Display the mapped U-values with index access
for index, u_value in enumerate(u_value_list):
    print(f"Index: {index}, Material: {material_keys[index]}, U-value: {u_value}")

Index: 0, Material: グラスウール断熱材通常品, U-value: 0.05
Index: 1, Material: グラスウール断熱材高性能品, U-value: 0.047
Index: 2, Material: 吹込み用グラスウール断熱材, U-value: 0.052
Index: 3, Material: ロックウール断熱材, U-value: 0.039
Index: 4, Material: 吹込み用ロックウール断熱材, U-value: 0.047
Index: 5, Material: 吹付けロックウール, U-value: 0.064
Index: 6, Material: 吹込み用セルローズファイバー断熱材, U-value: 0.04
Index: 7, Material: 押出法ポリスチレンフォーム断熱材, U-value: 0.04
Index: 8, Material: ポリエチレンフォーム断熱材, U-value: 0.042
Index: 9, Material: ビーズ法ポリスチレンフォーム断熱材, U-value: 0.041
Index: 10, Material: 硬質ウレタンフォーム断熱材, U-value: 0.029
Index: 11, Material: 吹付け硬質ウレタンフォーム, U-value: 0.04
Index: 12, Material: フェノールフォーム断熱材, U-value: 0.036
Index: 13, Material: インシュレーションファイバー断熱材, U-value: 0.052
Index: 14, Material: グラスウール断熱材１０K, U-value: 0.05
Index: 15, Material: グラスウール断熱材１２K, U-value: 0.045
Index: 16, Material: グラスウール断熱材１６K, U-value: 0.045
Index: 17, Material: グラスウール断熱材２０K, U-value: 0.042
Index: 18, Material: グラスウール断熱材２４K, U-value: 0.038
Index: 19, Material: グラスウール断熱材３２K, U-value: 0.

In [108]:
for ahu_name in inputdata["AirHandlingSystem"]:

    # 空調機タイプ（1つでも空調機があれば「空調機」と判断する）
    inputdata["AirHandlingSystem"][ahu_name]["AHU_type"] = "空調機以外"
    for unit_id, unit_configure in enumerate(
        inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    ):
        if unit_configure["Type"] == "空調機":
            inputdata["AirHandlingSystem"][ahu_name]["AHU_type"] = "空調機"
            break

    # 空調機の能力
    inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityCooling"] = 0
    inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityHeating"] = 0
    for unit_id, unit_configure in enumerate(
        inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    ):
        if unit_configure["RatedCapacityCooling"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityCooling"] += (
                unit_configure["RatedCapacityCooling"] * unit_configure["Number"]
            )

        if unit_configure["RatedCapacityHeating"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["RatedCapacityHeating"] += (
                unit_configure["RatedCapacityHeating"] * unit_configure["Number"]
            )

    # 空調機の風量 [m3/h]
    inputdata["AirHandlingSystem"][ahu_name]["FanAirVolume"] = 0
    for unit_id, unit_configure in enumerate(
        inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    ):
        if unit_configure["FanAirVolume"] != None:
            inputdata["AirHandlingSystem"][ahu_name]["FanAirVolume"] += (
                unit_configure["FanAirVolume"] * unit_configure["Number"]
            )

    # 全熱交換器の効率（一番低いものを採用）
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioCooling"] = None
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"] = None
    for unit_id, unit_configure in enumerate(inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]):
        # print(inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangeRatioHeating"])
        # 冷房の効率
        if unit_configure["AirHeatExchangeRatioCooling"] != None:

            if (
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioCooling"
                ]
                == None
            ):
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioCooling"
                ] = unit_configure["AirHeatExchangeRatioCooling"]
            elif (
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioCooling"
                ]
                > unit_configure["AirHeatExchangeRatioCooling"]
            ):
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioCooling"
                ] = unit_configure["AirHeatExchangeRatioCooling"]

        # 暖房の効率
        if unit_configure["AirHeatExchangeRatioHeating"] != None:
            if (
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioHeating"
                ]
                == None
            ):
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioHeating"
                ] = unit_configure["AirHeatExchangeRatioHeating"]
            elif (
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioHeating"
                ]
                > unit_configure["AirHeatExchangeRatioHeating"]
            ):
                inputdata["AirHandlingSystem"][ahu_name][
                    "AirHeatExchangeRatioHeating"
                ] = unit_configure["AirHeatExchangeRatioHeating"]

    # 全熱交換器のバイパス制御の有無（1つでもあればバイパス制御「有」とする）
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerControl"] = "無"
    # for unit_id, unit_configure in enumerate(
    #     inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    # ):
    #     if (unit_configure["AirHeatExchangeRatioCooling"] != None) and (
    #         unit_configure["AirHeatExchangeRatioHeating"] != None
    #     ):
    #         if unit_configure["AirHeatExchangerControl"] == "有":
    #             inputdata["AirHandlingSystem"][ahu_name][
    #                 "AirHeatExchangerControl"
    #             ] = "有"
    #             print("ari")

    # 全熱交換器の消費電力 [kW]
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerPowerConsumption"] = 0
    # for unit_id, unit_configure in enumerate(
    #     inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    # ):
    #     print(unit_configure["AirHeatExchangerPowerConsumption"])
    #     if unit_configure["AirHeatExchangerPowerConsumption"] != None:
    #         inputdata["AirHandlingSystem"][ahu_name][
    #             "AirHeatExchangerPowerConsumption"
    #         ] += (
    #             unit_configure["AirHeatExchangerPowerConsumption"]
    #             * unit_configure["Number"]
    #         )

    # 全熱交換器の風量 [m3/h]
    inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerAirVolume"] = 0
    for unit_id, unit_configure in enumerate(
        inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    ):
        if (unit_configure["AirHeatExchangeRatioCooling"] != None) and (
            unit_configure["AirHeatExchangeRatioHeating"] != None
        ):
            inputdata["AirHandlingSystem"][ahu_name][
                "AirHeatExchangerAirVolume"
            ] += (unit_configure["FanAirVolume"] * unit_configure["Number"])

In [109]:
for ahu_name in inputdata["AirHandlingSystem"]:
    # print(inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerPowerConsumption"])
    # print(inputdata["AirHandlingSystem"][ahu_name]["AirHeatExchangerAirVolume"])
    for unit_id, unit_configure in enumerate(
        inputdata["AirHandlingSystem"][ahu_name]["AirHandlingUnit"]
    ):
        print("定格冷却 = ", unit_configure["RatedCapacityCooling"])
        print("定格暖房 = ", unit_configure["RatedCapacityHeating"])
        print("台数 = ", unit_configure["Number"])
        print("全熱交換器の設計風量 = ", unit_configure["FanAirVolume"])
        print("全熱交換効率（冷，暖） = ", unit_configure["AirHeatExchangeRatioCooling"])


定格冷却 =  5.6
定格暖房 =  6.3
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  None
定格暖房 =  None
台数 =  1.0
全熱交換器の設計風量 =  200.0
全熱交換効率（冷，暖） =  60.0
定格冷却 =  3.6
定格暖房 =  4.0
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  None
定格暖房 =  None
台数 =  1.0
全熱交換器の設計風量 =  150.0
全熱交換効率（冷，暖） =  60.0
定格冷却 =  3.6
定格暖房 =  4.0
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  None
定格暖房 =  None
台数 =  1.0
全熱交換器の設計風量 =  150.0
全熱交換効率（冷，暖） =  60.0
定格冷却 =  3.6
定格暖房 =  4.0
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  None
定格暖房 =  None
台数 =  1.0
全熱交換器の設計風量 =  300.0
全熱交換効率（冷，暖） =  60.0
定格冷却 =  6.79
定格暖房 =  6.77
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  2.7
定格暖房 =  2.62
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  2.7
定格暖房 =  2.62
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  2.7
定格暖房 =  2.62
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  2.7
定格暖房 =  2.62
台数 =  1.0
全熱交換器の設計風量 =  None
全熱交換効率（冷，暖） =  None
定格冷却 =  2.7
定格暖房 =  2.62
台数 =  1.

In [110]:
# 二次ポンプが空欄であった場合、ダミーの仮想ポンプを追加する。
number = 0
for ahu_name in inputdata["AirHandlingSystem"]:

    if inputdata["AirHandlingSystem"][ahu_name]["Pump_cooling"] == None:

        inputdata["AirHandlingSystem"][ahu_name]["Pump_cooling"] = (
            "dummyPump_" + str(number)
        )

        inputdata["SecondaryPumpSystem"]["dummyPump_" + str(number)] = {
            "冷房": {
                "TemperatureDifference": 0,
                "isStagingControl": "無",
                "SecondaryPump": [
                    {
                        "Number": 0,
                        "RatedWaterFlowRate": 0,
                        "RatedPowerConsumption": 0,
                        "ContolType": "無",
                        "MinOpeningRate": 100,
                    }
                ],
            }
        }

        number += 1

    if inputdata["AirHandlingSystem"][ahu_name]["Pump_heating"] == None:

        inputdata["AirHandlingSystem"][ahu_name]["Pump_heating"] = (
            "dummyPump_" + str(number)
        )

        inputdata["SecondaryPumpSystem"]["dummyPump_" + str(number)] = {
            "暖房": {
                "TemperatureDifference": 0,
                "isStagingControl": "無",
                "SecondaryPump": [
                    {
                        "Number": 0,
                        "RatedWaterFlowRate": 0,
                        "RatedPowerConsumption": 0,
                        "ContolType": "無",
                        "MinOpeningRate": 100,
                    }
                ],
            }
        }

        number += 1

In [111]:

for ahu_name in inputdata["AirHandlingSystem"]:
    inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply"] = "無"
    inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply_cooling"] = "無"
    inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply_heating"] = "無"
for pump_name in inputdata["SecondaryPumpSystem"]:
    inputdata["SecondaryPumpSystem"][pump_name]["isSimultaneousSupply"] = "無"
for ref_name in inputdata["HeatsourceSystem"]:
    inputdata["HeatsourceSystem"][ref_name]["isSimultaneousSupply"] = "無"


# 両方とも冷暖同時なら、その空調機群は冷暖同時運転可能とする。
for ahu_name in inputdata["AirHandlingSystem"]:

    if (
        inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply_cooling"]
        == "有"
    ) and (
        inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply_heating"]
        == "有"
    ):

        inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply"] = "有"

In [112]:
# 両方とも冷暖同時なら、その空調機群は冷暖同時運転可能とする。
for ahu_name in inputdata["AirHandlingSystem"]:

    print(inputdata["AirHandlingSystem"][ahu_name]["isSimultaneousSupply_cooling"])


無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
無
